In [1]:
# reflected location error in ozone data simulation

import torch
import torch.fft
import numpy as np
import sys
import os
import logging
import argparse 
import pandas as pd
import pickle
import torch.optim as optim
import copy
import time
from sklearn.neighbors import BallTree
from typing import Optional, List, Tuple
from pathlib import Path

# --- CUSTOM PATHS ---
gems_tco_path = "/Users/joonwonlee/Documents/GEMS_TCO-1/src"
sys.path.append(gems_tco_path)

# (필요 시 실제 GEMS_TCO 라이브러리 import)
try:
    from GEMS_TCO import kernels_reparam_space_time_gpu
    from GEMS_TCO import kernels_reparam_space_time_gpu_copy_dummy_013126 as kernels_reparam_space_time_gpu_copy
    from GEMS_TCO import kernels_columns as kernels_reparam_space_time_gpu_col
    
    from GEMS_TCO import orderings as _orderings
    from GEMS_TCO import alg_optimization, BaseLogger


except ImportError:
    print("Warning: GEMS_TCO modules not found. Ensure paths are correct.")


from GEMS_TCO import configuration as config
from GEMS_TCO.data_loader import load_data2, exact_location_filter
from GEMS_TCO import debiased_whittle





In [7]:
space: List[str] = ['1', '1']
lat_lon_resolution = [int(s) for s in space]
mm_cond_number: int = 8
years = ['2024']
month_range = [7] 

output_path = input_path = Path(config.mac_estimates_day_path)
data_load_instance = load_data2(config.mac_data_load_path)

#lat_range_input = [1, 3]
#lon_range_input = [125.0, 129.0]

lat_range_input=[-3,-1]      
lon_range_input=[121, 125] 

#lat_range_input=[-3,2]      
#lon_range_input=[121, 131] 

df_map, ord_mm, nns_map = data_load_instance.load_maxmin_ordered_data_bymonthyear(
lat_lon_resolution=lat_lon_resolution, 
mm_cond_number=mm_cond_number,
years_=years, 
months_=month_range,

lat_range=lat_range_input,   
lon_range=lon_range_input
  
)


daily_aggregated_reg_vecc_sim = [] 
daily_hourly_maps_reg_vecc_sim = []      

daily_aggregated_irr_vecc_sim = [] 
daily_hourly_maps_irr_vecc_sim = []   


for day_index in range(31):
    hour_start_index = day_index * 8
    
    hour_end_index = (day_index + 1) * 8
    #hour_end_index = day_index*8 + 1
    hour_indices = [hour_start_index, hour_end_index]

    day_hourly_map, day_aggregated_tensor = data_load_instance.load_working_data(
    df_map, 
    hour_indices, 
    ord_mm= ord_mm,  # or just omit it
    dtype=torch.float64, # or just omit it 
    keep_ori=False  #keep_exact_loc
    )

    daily_aggregated_reg_vecc_sim.append( day_aggregated_tensor )
    daily_hourly_maps_reg_vecc_sim.append( day_hourly_map )

    day_hourly_map, day_aggregated_tensor = data_load_instance.load_working_data(
    df_map, 
    hour_indices, 
    ord_mm= ord_mm,  # or just omit it
    dtype=torch.float64, # or just omit it 
    keep_ori=True  #keep_exact_loc
    )

    daily_aggregated_irr_vecc_sim.append( day_aggregated_tensor )
    daily_hourly_maps_irr_vecc_sim.append( day_hourly_map )
print(daily_aggregated_irr_vecc_sim[0].shape)

nn = daily_aggregated_irr_vecc_sim[0].shape[0]


torch.Size([22680, 4])


In [8]:
import torch
import torch.nn.functional as F
import numpy as np
from sklearn.neighbors import BallTree
import sys


# --- 2. EXACT COVARIANCE & FFT HELPERS ---
def get_model_covariance_on_grid(lags_x, lags_y, lags_t, params):
    phi1, phi2, phi3, phi4 = torch.exp(params[0]), torch.exp(params[1]), torch.exp(params[2]), torch.exp(params[3])
    advec_lat, advec_lon = params[4], params[5]
    sigmasq = phi1 / phi2

    u_lat_eff = lags_x - advec_lat * lags_t
    u_lon_eff = lags_y - advec_lon * lags_t
    dist_sq = (u_lat_eff.pow(2) * phi3) + (u_lon_eff.pow(2)) + (lags_t.pow(2) * phi4)
    return sigmasq * torch.exp(-torch.sqrt(dist_sq + 1e-8) * phi2)


def make_target_grid(lat_start, lat_end, lat_step, lon_start, lon_end, lon_step, device, dtype):
    """
    표준 분석용 레귤러 그리드 생성. 
    자동 방향 제어(아래 버전 장점) + 소수점 반올림 오차 방지(위 버전 장점) 통합.
    """
    # 1. 방향 자동 교정 (Step 방향이 시작/끝점과 맞지 않을 경우)
    if lat_start > lat_end and lat_step > 0: 
        lat_step = -lat_step
    if lon_start > lon_end and lon_step > 0:
        lon_step = -lon_step

    # 2. Arange 생성
    lats = torch.arange(lat_start, lat_end - 0.0001, lat_step, device=device, dtype=dtype)
    lons = torch.arange(lon_start, lon_end + 0.0001, lon_step, device=device, dtype=dtype)

    # 3. 수치적 안정성을 위한 Rounding (소수점 4자리)
    # 이 과정이 없으면 나중에 모델 피팅 시 좌표 미세 오차로 인해 결과가 달라질 수 있습니다.
    lats = torch.round(lats * 10000) / 10000
    lons = torch.round(lons * 10000) / 10000

    # 4. Meshgrid 생성
    grid_lat, grid_lon = torch.meshgrid(lats, lons, indexing='ij')
    center_points = torch.stack([grid_lat.flatten(), grid_lon.flatten()], dim=1)
    
    return center_points, len(lats), len(lons)

# --- 2. High-Res Field Generator (Lat x160 / Lon x4) ---
def generate_high_res_field(target_lat_range, target_lon_range, t_steps, params, device, dtype):
    """
    초고해상도(160배/4배) 필드를 생성하고, 분산을 참값(Target)에 맞게 보정(Calibration)합니다.
    """
    # [설정] 해상도 확대 배율
    lat_res_factor = 160.0  # 위도 160배
    lon_res_factor = 4.0    # 경도 4배
    
    # 기본 해상도(0.044, 0.063)를 배율로 나눔
    lat_res_high = 0.044 / lat_res_factor
    lon_res_high = 0.063 / lon_res_factor
    
    t_lat_max = max(target_lat_range)
    t_lat_min = min(target_lat_range)
    
    # Padding: 경계면 왜곡 방지를 위해 약간 넓게 생성
    lats_high = torch.arange(t_lat_max + 0.1, t_lat_min - 0.1, -lat_res_high, device=device, dtype=dtype)
    lons_high = torch.arange(target_lon_range[0] - 0.1, target_lon_range[1] + 0.1, lon_res_high, device=device, dtype=dtype)
    
    Nx, Ny, Nt = len(lats_high), len(lons_high), t_steps
    dlat, dlon, dt = lat_res_high, lon_res_high, 1.0
    
    # FFT Grid Setup (Circulant Embedding)
    Px, Py, Pt = 2*Nx, 2*Ny, 2*Nt
    lags_x = torch.arange(Px, device=device, dtype=dtype) * dlat; lags_x[Px//2:] -= (Px * dlat)
    lags_y = torch.arange(Py, device=device, dtype=dtype) * dlon; lags_y[Py//2:] -= (Py * dlon)
    lags_t = torch.arange(Pt, device=device, dtype=dtype) * dt;   lags_t[Pt//2:] -= (Pt * dt)

    L_x, L_y, L_t = torch.meshgrid(lags_x, lags_y, lags_t, indexing='ij')
    
    # 파라미터 해석
    phi1, phi2 = torch.exp(params[0]), torch.exp(params[1])
    phi3, phi4 = torch.exp(params[2]), torch.exp(params[3])
    adv_lat, adv_lon = params[4], params[5]
    
    sigma_sq = phi1 / phi2  # 참값 분산 (13.059)
    
    u_x = L_x - adv_lat * L_t
    u_y = L_y - adv_lon * L_t
    dist_sq = (u_x * torch.sqrt(phi3) * phi2)**2 + (u_y * phi2)**2 + (L_t * torch.sqrt(phi4) * phi2)**2
    C_vals = sigma_sq * torch.exp(-torch.sqrt(dist_sq + 1e-12))

    # FFT Simulation
    S = torch.fft.fftn(C_vals); S.real = torch.clamp(S.real, min=0)
    random_phase = torch.fft.fftn(torch.randn(Px, Py, Pt, device=device, dtype=dtype))
    field_sim_raw = torch.fft.ifftn(torch.sqrt(S.real) * random_phase).real
    
    # Crop
    field_sim = field_sim_raw[:Nx, :Ny, :Nt]
    
    # [Calibration] 160배 해상도로 인해 흩어진 에너지를 모아 분산을 복원
    current_std = field_sim.std()
    target_std = torch.sqrt(sigma_sq)
    
    # (x - mean) * ratio
    field_calibrated = (field_sim - field_sim.mean()) * (target_std / (current_std + 1e-9))
    
    print(f"   [High-Res Gen] 160x/4x Field Created.")
    print(f"   [Calibration] Raw Std: {current_std.item():.4f} -> Fixed to: {target_std.item():.4f}")
    
    return field_calibrated, lats_high, lons_high

# --- 3. Main Generator (Vecc & DW) ---
# ==============================================================================
# 1. 수정된 데이터셋 생성기 (Size Mismatch 해결 버전)
# ==============================================================================

def generate_exact_count_datasets_fixed(daily_maps_real, true_params_tensor, target_grid_info, device, dtype):
    """
    Step 1에서 로드된 22,680개의 구조를 그대로 유지하며 값만 시뮬레이션으로 채웁니다.
    """
    lat_s, lat_e, lat_step, lon_s, lon_e, lon_step = target_grid_info
    
    # [A] 타겟 격자 준비
    target_grid_coords, Nx, Ny = make_target_grid(lat_s, lat_e, lat_step, lon_s, lon_e, lon_step, device, dtype)
    target_tree = BallTree(np.radians(target_grid_coords.cpu().numpy()), metric='haversine') 
    
    # [B] High-Res Field 준비
    high_res_field, lats_high, lons_high = generate_high_res_field((lat_s, lat_e), (lon_s, lon_e), 8, true_params_tensor, device, dtype)
    hr_mesh_lat, hr_mesh_lon = torch.meshgrid(lats_high, lons_high, indexing='ij')
    hr_tree = BallTree(np.radians(torch.stack([hr_mesh_lat.flatten(), hr_mesh_lon.flatten()], dim=1).cpu().numpy()), metric='haversine')
    high_res_flat = high_res_field.reshape(-1, 8) 

    irr_map_dict, dw_map_dict = {}, {}
    irr_list, dw_list = [], []
    noise_std = torch.sqrt(torch.exp(true_params_tensor[6]))

    # [C] 데이터 매핑 (구조 보존)
    # real_maps_list[0] (로드된 22680개 데이터) 사용
    day0_dict = daily_maps_real[0]
    sorted_keys = sorted([k for k in day0_dict.keys() if 'hm' in k or 'time' in k])
    
    for t_idx, key in enumerate(sorted_keys):
        if t_idx >= 8: break
        real_tensor = day0_dict[key].to(device)
        
        # ------------------------------------------------------------------
        # [수정 핵심] clean_tensor 로직 제거 -> 원본 real_tensor(2835개/시간) 그대로 사용
        # ------------------------------------------------------------------
        clean_tensor = real_tensor 
        real_locs = clean_tensor[:, :2]
        
        # High-Res Field에서 값 샘플링
        _, hr_indices = hr_tree.query(np.radians(real_locs.cpu().numpy()), k=1)
        signal = high_res_flat[torch.tensor(hr_indices.flatten(), device=device), t_idx]
        sim_vals = signal + (torch.randn_like(signal) * noise_std) + 260.0
        
        # 1. Vecc (Irregular): 원본 위치 유지
        vecc_row = torch.stack([real_locs[:,0], real_locs[:,1], sim_vals, clean_tensor[:,3]], dim=1).detach()
        irr_map_dict[key] = vecc_row
        irr_list.append(vecc_row)
        
        # 2. DW (Regular): 격자 중앙으로 매핑
        # 여기서 target_grid_coords와 real_locs의 개수가 1:1 대응되어야 함 (2835개)
        # 만약 target_grid_coords가 더 많다면 슬라이싱 처리
        current_grid_points = target_grid_coords[:len(clean_tensor)]
        
        dw_row = torch.stack([current_grid_points[:,0], current_grid_points[:,1], sim_vals, clean_tensor[:,3]], dim=1).detach()
        dw_map_dict[key] = dw_row
        dw_list.append(dw_row)

    # 2835 * 8 = 22680 유지 확인
    return [torch.cat(dw_list)], [dw_map_dict], [torch.cat(irr_list)], [irr_map_dict]
# --- 4. EXECUTION ---
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
DTYPE = torch.float64

target_grid_info = (-1.0, -3.0, 0.044, 121.0, 125.0, 0.063)

# 1. 참값 정의
init_sigmasq   = 13.059
init_range_lon = 0.195 
init_range_lat = 0.154 
init_range_time = 1.0
init_nugget    = 0.247

# 2. 파라미터 변환 (정답 로직)
true_phi2 = 1.0 / init_range_lon              
true_phi1 = init_sigmasq * true_phi2          
true_phi3 = (init_range_lon / init_range_lat)**2
true_phi4 = (init_range_lon / init_range_time)**2

true_params_tensor = [
    torch.tensor([np.log(true_phi1)], device=DEVICE, dtype=DTYPE),
    torch.tensor([np.log(true_phi2)], device=DEVICE, dtype=DTYPE),
    torch.tensor([np.log(true_phi3)], device=DEVICE, dtype=DTYPE),
    torch.tensor([np.log(true_phi4)], device=DEVICE, dtype=DTYPE),
    torch.tensor([0.0418], device=DEVICE, dtype=DTYPE),
    torch.tensor([-0.1689], device=DEVICE, dtype=DTYPE),
    torch.tensor([np.log(init_nugget)], device=DEVICE, dtype=DTYPE)
]

print(f"Checking Inputs:")
print(f" -> Target SigmaSq: {init_sigmasq}")

if 'daily_hourly_maps_irr_vecc_sim' in locals() and len(daily_hourly_maps_irr_vecc_sim) > 0:
    (daily_aggregated_reg_vecc_sim, daily_hourly_maps_reg_vecc_sim,
     daily_aggregated_irr_vecc_sim, daily_hourly_maps_irr_vecc_sim) = generate_exact_count_datasets_fixed(
        daily_hourly_maps_irr_vecc_sim, true_params_tensor, target_grid_info, DEVICE, DTYPE
    )
    
    # 검증: Vecc vs DW
    v_agg = daily_aggregated_irr_vecc_sim[0]
    dw_agg = daily_aggregated_reg_vecc_sim[0]
    
    print(f"\n[Final Validation]")
    print(f"Vecc Shape (Irregular): {v_agg.shape}")
    print(f"DW Shape   (Regular):   {dw_agg.shape}")
    print(f"Variance Check: {v_agg[:, 2].var().item():.4f} (Expected ~13.0 + Nugget)")

else:
    print("Error: Load data first.")

Checking Inputs:
 -> Target SigmaSq: 13.059
   [High-Res Gen] 160x/4x Field Created.
   [Calibration] Raw Std: 3.4647 -> Fixed to: 3.6137

[Final Validation]
Vecc Shape (Irregular): torch.Size([22680, 4])
DW Shape   (Regular):   torch.Size([22680, 4])
Variance Check: 13.2800 (Expected ~13.0 + Nugget)


dummy time


In [3]:
import torch
import torch.nn.functional as F
import numpy as np
from sklearn.neighbors import BallTree
import time
import sys
import typer

# --- Helper Functions ---

def make_target_grid(lat_start, lat_end, lat_step, lon_start, lon_end, lon_step, device, dtype):
    """Target Regular Grid 생성"""
    if lat_start > lat_end and lat_step > 0:
        lat_step = -lat_step
        
    lats = torch.arange(lat_start, lat_end - 0.0001, lat_step, device=device, dtype=dtype)
    lons = torch.arange(lon_start, lon_end - 0.0001, lon_step, device=device, dtype=dtype)
    
    grid_lat, grid_lon = torch.meshgrid(lats, lons, indexing='ij')
    center_points = torch.stack([grid_lat.flatten(), grid_lon.flatten()], dim=1)
    
    return center_points, len(lats), len(lons)

def generate_high_res_field(target_lat_range, target_lon_range, t_steps, params, device, dtype):
    """3배 고해상도 True Field 생성"""
    res_factor = 3.0
    lat_res_high = 0.044 / res_factor
    lon_res_high = 0.063 / res_factor
    
    pad = 0.2
    lats_high = torch.arange(target_lat_range[0] + pad, target_lat_range[1] - pad, -lat_res_high, device=device, dtype=dtype)
    lons_high = torch.arange(target_lon_range[0] - pad, target_lon_range[1] + pad, lon_res_high, device=device, dtype=dtype)
    
    Nx, Ny, Nt = len(lats_high), len(lons_high), t_steps
    dlat, dlon, dt = lat_res_high, lon_res_high, 1.0
    
    Px, Py, Pt = 2*Nx, 2*Ny, 2*Nt
    lags_x = torch.arange(Px, device=device, dtype=dtype) * dlat
    lags_x[Px//2:] -= (Px * dlat)
    lags_y = torch.arange(Py, device=device, dtype=dtype) * dlon
    lags_y[Py//2:] -= (Py * dlon)
    lags_t = torch.arange(Pt, device=device, dtype=dtype) * dt
    lags_t[Pt//2:] -= (Pt * dt)

    L_x, L_y, L_t = torch.meshgrid(lags_x, lags_y, lags_t, indexing='ij')

    phi1, phi2 = torch.exp(params[0]), torch.exp(params[1])
    phi3, phi4 = torch.exp(params[2]), torch.exp(params[3])
    adv_lat, adv_lon = params[4], params[5]
    sigmasq = phi1 / phi2
    
    u_x = L_x - adv_lat * L_t
    u_y = L_y - adv_lon * L_t
    dist_sq = (u_x * torch.sqrt(phi3) * phi2)**2 + (u_y * phi2)**2 + (L_t * torch.sqrt(phi4) * phi2)**2
    C_vals = sigmasq * torch.exp(-torch.sqrt(dist_sq + 1e-12))

    S = torch.fft.fftn(C_vals)
    S.real = torch.clamp(S.real, min=0)
    random_phase = torch.fft.fftn(torch.randn(Px, Py, Pt, device=device, dtype=dtype))
    field_sim = torch.fft.ifftn(torch.sqrt(S.real) * random_phase).real
    
    return field_sim[:Nx, :Ny, :Nt], lats_high, lons_high

# --- Core Generator (12 Columns Logic) ---

def generate_exact_count_datasets(
    daily_maps_real,    
    true_params_tensor, 
    target_grid_info,   
    device, dtype
):
    print("--- Generating Datasets (12 Cols: Lat, Lon, Val, Time, H0..H7) ---")
    
    lat_s, lat_e, lat_step, lon_s, lon_e, lon_step = target_grid_info
    
    # 1. Target Grid Coords
    target_grid_coords, _, _ = make_target_grid(
        lat_s, lat_e, lat_step, lon_s, lon_e, lon_step, device, dtype
    )
    
    target_locs_np = target_grid_coords.cpu().numpy()
    target_locs_rad = np.radians(target_locs_np)
    
    def normalize(val, min_v, max_v):
        return 2 * (val - min_v) / (max_v - min_v) - 1

    daily_aggregated_tensors_vecc_sim = []
    daily_hourly_maps_vecc_sim = []
    daily_aggregated_tensors_dw_sim = []
    daily_hourly_maps_dw_sim = []
    
    # --- Day Loop ---
    for day_idx, day_map_real in enumerate(daily_maps_real):
        
        # (A) High-Res Truth
        high_res_field, lats_high, lons_high = generate_high_res_field(
            (lat_s, lat_e), (lon_s, lon_e), 8, true_params_tensor, device, dtype
        )
        
        lat_max_b, lat_min_b = lats_high[0], lats_high[-1]
        lon_min_b, lon_max_b = lons_high[0], lons_high[-1]
        
        # (B) Hour Loop
        sorted_keys = sorted([k for k in day_map_real.keys() if 'hm' in k or 'time' in k])
        
        current_day_vecc_map = {}
        current_day_dw_map = {}
        
        vecc_tensor_list = []
        dw_tensor_list = []
        
        for t_idx, key in enumerate(sorted_keys):
            if t_idx >= 8: break
            
            # 1. Extract Real Info
            real_tensor = day_map_real[key]
            if not isinstance(real_tensor, torch.Tensor):
                real_tensor = torch.tensor(real_tensor, device=device, dtype=dtype)
            else:
                real_tensor = real_tensor.to(device)
            
            real_lats = real_tensor[:, 0]
            real_lons = real_tensor[:, 1]
            time_val_continuous = real_tensor[:, 3] # [N] (e.g., 21.0, 22.0)
            
            N_points = real_lats.shape[0]
            
            # [NEW] Create 8 Dummy Variables
            dummy_time = torch.zeros((N_points, 8), device=device, dtype=dtype)
            dummy_time[:, t_idx] = 1.0 
            
            # 2. Sample Values
            norm_lons = normalize(real_lons, lon_min_b, lon_max_b)
            norm_lats = normalize(real_lats, lat_min_b, lat_max_b)
            
            grid_sample_input = torch.stack([norm_lons, norm_lats], dim=-1).unsqueeze(0).unsqueeze(0)
            field_slice = high_res_field[:, :, t_idx].unsqueeze(0).unsqueeze(0)
            
            sim_vals = F.grid_sample(field_slice, grid_sample_input, mode='bilinear', align_corners=True).squeeze()
            sim_vals = sim_vals + 260.0 
            
            # 3. [Set 1: VECC SIM] - Irregular
            # Col 0: Lat
            # Col 1: Lon
            # Col 2: Val
            # Col 3: Time (Continuous) -> For Kernel
            # Col 4-11: H0~H7 (Dummy) -> For Mean
            vecc_row = torch.cat([
                real_lats.unsqueeze(1), 
                real_lons.unsqueeze(1), 
                sim_vals.unsqueeze(1), 
                time_val_continuous.unsqueeze(1), 
                dummy_time
            ], dim=1)
            
            current_day_vecc_map[key] = vecc_row.detach()
            vecc_tensor_list.append(vecc_row.detach())
            
            # 4. [Set 2: DW SIM] - Regular Mapping
            real_locs_np = torch.stack([real_lats, real_lons], dim=1).cpu().numpy()
            
            tree = BallTree(target_locs_rad, metric='haversine') 
            _, indices = tree.query(np.radians(real_locs_np), k=1)
            indices = torch.tensor(indices.flatten(), device=device, dtype=torch.long)
            
            mapped_lats = target_grid_coords[indices, 0]
            mapped_lons = target_grid_coords[indices, 1]
            
            # Time & Dummies 복사 (Mapped된 위치에도 동일하게 적용)
            mapped_time_val = time_val_continuous # 시간은 모두 같음
            mapped_dummies = dummy_time # 더미도 모두 같음
            
            dw_row = torch.cat([
                mapped_lats.unsqueeze(1), 
                mapped_lons.unsqueeze(1), 
                sim_vals.unsqueeze(1), 
                mapped_time_val.unsqueeze(1),
                mapped_dummies
            ], dim=1)
            
            current_day_dw_map[key] = dw_row.detach()
            dw_tensor_list.append(dw_row.detach())

        # (C) Aggregate
        daily_hourly_maps_vecc_sim.append(current_day_vecc_map)
        daily_aggregated_tensors_vecc_sim.append(torch.cat(vecc_tensor_list, dim=0))
        
        daily_hourly_maps_dw_sim.append(current_day_dw_map)
        daily_aggregated_tensors_dw_sim.append(torch.cat(dw_tensor_list, dim=0))
        
        if (day_idx + 1) % 5 == 0:
            print(f"  Processed Day {day_idx+1}")
            
    return (daily_aggregated_tensors_dw_sim, daily_hourly_maps_dw_sim,
            daily_aggregated_tensors_vecc_sim, daily_hourly_maps_vecc_sim)

# --- Execution ---

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
DTYPE = torch.float64

target_grid_info = (2.0, -3.0, 0.044, 121.0, 131.0, 0.063)
true_params_tensor = [torch.tensor([np.log(10.0)], device=DEVICE, dtype=DTYPE) for _ in range(7)]

if 'daily_hourly_maps_vecc' in locals() and len(daily_hourly_maps_vecc) > 0:
    
    (daily_aggregated_tensors_dw_sim, 
     daily_hourly_maps_dw_sim,
     daily_aggregated_tensors_vecc_sim, 
     daily_hourly_maps_vecc_sim) = generate_exact_count_datasets(
        daily_hourly_maps_vecc,
        true_params_tensor,
        target_grid_info,
        DEVICE, DTYPE
    )
    
    # [Final Verification]
    v_tensor = daily_aggregated_tensors_vecc_sim[0]
    print("\n[Verification]")
    print(f"Dataset Shape: {v_tensor.shape}")
    
    if v_tensor.shape[1] == 12:
        print("✅ Success: Output has 12 columns (Lat, Lon, Val, Time, + 8 Dummies)")
        # 예시: 3번째 컬럼은 Continuous Time, 4번째부터는 Dummy
        print(f"Sample Row: \n{v_tensor[0]}")
    else:
        print(f"❌ Error: Expected 12 columns, got {v_tensor.shape[1]}")

else:
    print("❌ Error: 'daily_hourly_maps_vecc' is not loaded.")

--- Generating Datasets (12 Cols: Lat, Lon, Val, Time, H0..H7) ---
  Processed Day 5
  Processed Day 10
  Processed Day 15
  Processed Day 20
  Processed Day 25
  Processed Day 30

[Verification]
Dataset Shape: torch.Size([22680, 12])
✅ Success: Output has 12 columns (Lat, Lon, Val, Time, + 8 Dummies)
Sample Row: 
tensor([ -1.9987, 122.9810, 258.8952,  21.0000,   1.0000,   0.0000,   0.0000,
          0.0000,   0.0000,   0.0000,   0.0000,   0.0000], dtype=torch.float64)


set up

Likelihood vs. Truth: A model with the wrong parameters (short range) might produce a higher Vecchia likelihood because it fits the high-frequency noise better, but it will be terrible at prediction (Kriging) away from data points.

# Fit vecchia max min time 2 

vecc 

In [9]:
import torch
import numpy as np
import time

# --- CONFIGURATION ---
v = 0.5              # Smoothness
mm_cond_number = 8   # Neighbors
#mm_cond_number = 16   # Neighbors
nheads = 300           # 0 = Pure Vecchia
lr = 1.0             # LBFGS learning rate
LBFGS_MAX_STEPS = 3
LBFGS_HISTORY_SIZE = 100 # 100
LBFGS_LR = 1.0
LBFGS_MAX_EVAL = 30    

#DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {DEVICE}")

# --- 1. SETUP PARAMETERS (List of Scalars) ---
# Truth: [4.18, 1.94, 0.24, -3.97, 0.014, -0.20, -0.85]
init_sigmasq   = 13.059
init_range_lat = 0.154 
init_range_lon = 0.195
init_advec_lat = 0.0218
init_range_time = 1.0
init_advec_lon = -0.1689
init_nugget    = 0.247

# Map model parameters to the 'phi' reparameterization
init_phi2 = 1.0 / init_range_lon                # 1/range_lon
init_phi1 = init_sigmasq * init_phi2            # sigmasq / range_lon
init_phi3 = (init_range_lon / init_range_lat)**2  # (range_lon / range_lat)^2
init_phi4 = (init_range_lon / init_range_time)**2      # (range_lon / range_time)^2

# Create Initial Parameters (Float64, Requires Grad)
initial_vals = [np.log(init_phi1), np.log(init_phi2), np.log(init_phi3), 
                np.log(init_phi4), init_advec_lat, init_advec_lon, np.log(init_nugget)]

# [4.2042, 1.6348, 0.4721, -3.2695, 0.0218, -0.1689, -1.3984]
params_list = [
    torch.tensor([val], requires_grad=True, dtype=torch.float64, device=DEVICE)
    for val in initial_vals
]

# --- 2. INSTANTIATE MODEL ---
print(f'\n{"="*40}')
print(f'--- Initializing VecchiaBatched Model ---')
print(f'{"="*40}')

if isinstance(daily_aggregated_irr_vecc_sim, torch.Tensor):
    daily_aggregated_irr_vecc_sim = daily_aggregated_irr_vecc_sim.to(DEVICE)

# Instantiate
model_instance = kernels_reparam_space_time_gpu_copy.fit_vecchia_lbfgs(
    smooth=v,
    #input_map=daily_hourly_maps_vecc_sim[0],
    #aggregated_data= daily_aggregated_tensors_vecc_sim[0],

    input_map=daily_hourly_maps_irr_vecc_sim[0],
    aggregated_data= daily_aggregated_irr_vecc_sim[0],
    nns_map=nns_map,
    mm_cond_number=mm_cond_number,
    nheads=nheads
)

'''
model_instance = kernels_reparam_space_time_gpu_col.fit_vecchia_lbfgs(
    smooth=v,
    #input_map=daily_hourly_maps_vecc_sim[0],
    #aggregated_data= daily_aggregated_tensors_vecc_sim[0],

    input_map=daily_hourly_maps_irr_vecc_sim[0],
    aggregated_data= daily_aggregated_irr_vecc_sim[0],

    nns_map=None,
    mm_cond_number=mm_cond_number
)
''' 

# --- 3. OPTIMIZATION LOOP ---
print(f'\n{"="*40}')
print(f'--- Running L-BFGS Optimization ---')
print(f'{"="*40}')

# Optimizer takes the LIST of scalars
optimizer_vecc = model_instance.set_optimizer(
            params_list,     
            lr=LBFGS_LR,            
            max_iter=LBFGS_MAX_EVAL,        
            history_size=LBFGS_HISTORY_SIZE 
        )

start_time = time.time()

out, steps_ran = model_instance.fit_vecc_lbfgs(
        params_list,
        optimizer_vecc,
        # covariance_function argument is GONE
        max_steps=LBFGS_MAX_STEPS, 
        grad_tol=1e-7
    )


end_time = time.time()
epoch_time = end_time - start_time

print(f"\nOptimization finished in {epoch_time:.2f}s.")
print(f"Results after {steps_ran} steps: {out}")
print(f"Final Params: {torch.cat(params_list).detach().cpu().numpy()}")

Using device: cpu

--- Initializing VecchiaBatched Model ---

--- Running L-BFGS Optimization ---
🚀 Pre-computing (Corrected Vectorization)... ✅ Done in 0.1250s. (Heads: 2400, Tails: 20280)
--- Starting Batched L-BFGS Optimization (GPU) ---
--- Step 1/3 / Loss: 1.179761 ---
  Param 0: Value=4.2920, Grad=-4.679390900186407e-06
  Param 1: Value=1.6880, Grad=5.122242548370693e-07
  Param 2: Value=0.4883, Grad=2.8293805617383684e-06
  Param 3: Value=-3.1809, Grad=6.194590688192968e-07
  Param 4: Value=-0.0553, Grad=8.05757962505163e-06
  Param 5: Value=-0.1472, Grad=-4.232528960437798e-07
  Param 6: Value=-1.4506, Grad=-3.062077376579067e-07
  Max Abs Grad: 8.057580e-06
------------------------------
--- Step 2/3 / Loss: 1.177100 ---
  Param 0: Value=4.2920, Grad=-4.679390900186407e-06
  Param 1: Value=1.6880, Grad=5.122242548370693e-07
  Param 2: Value=0.4883, Grad=2.8293805617383684e-06
  Param 3: Value=-3.1809, Grad=6.194590688192968e-07
  Param 4: Value=-0.0553, Grad=8.05757962505163e-

In [ ]:
Final Interpretable Params: {'sigma_sq': 13.048806443143716, 'range_lon': 0.2759775584138689, 'range_lat': 0.26948439045514627, 'range_time': 1.2123116662337672, 'advec_lat': 0.003713422682726235, 'advec_lon': -0.18035976962575426, 'nugget': 4.6861836446362183e-08}
Final Interpretable Params: {'sigma_sq': 13.719100200518179, 'range_lon': 0.32214277436468963, 'range_lat': 0.32239904147765264, 'range_time': 1.4623993761954481, 'advec_lat': -0.03314564565869727, 'advec_lon': -0.25639209965916504, 'nugget': 0.38030028721574305}


기존 (Naive)	점 하나하나마다 매번 행렬을 만들고 분해함	20,000번	🐢 매우 느림
현재 (Grouped)	같은 모양을 가진 점끼리 모아서, 대표로 1번만 분해함	50번	🚀 400배

dw

Final Interpretable Params: {'sigma_sq': 13.048806443143716, 'range_lon': 0.2759775584138689, 'range_lat': 0.26948439045514627, 'range_time': 1.2123116662337672, 'advec_lat': 0.003713422682726235, 'advec_lon': -0.18035976962575426, 'nugget': 4.6861836446362183e-08}


dw_sim
Final Interpretable Params: {'sigma_sq': 11.817838113249758, 'range_lon': 0.22621206038334832, 'range_lat': 0.17422723320268027, 'range_time': 0.997943651842861, 'advec_lat': -0.050699303814975866, 'advec_lon': -0.18116598528943043, 'nugget': 1.1984570483507146}


vecc

Final Interpretable Params: {'sigma_sq': 13.719100200518179, 'range_lon': 0.32214277436468963, 'range_lat': 0.32239904147765264, 'range_time': 1.4623993761954481, 'advec_lat': -0.03314564565869727, 'advec_lon': -0.25639209965916504, 'nugget': 0.38030028721574305}


vecc_sim

Final Interpretable Params: {'sigma_sq': 12.967334679138654, 'range_lon': 0.1829644675670931, 'range_lat': 0.14094812906918186, 'range_time': 0.9219567362979819, 'advec_lat': -0.04561716644679645, 'advec_lon': -0.17524213405286115, 'nugget': 0.1315329566135699}






###

Final Interpretable Params: {
   'sigma_sq': 12.982146243206834, '
   range_lon': 0.1774660571793038, 
   'range_lat': 0.1377164149996857, 
   'range_time': 0.8960320754550594, 
   'advec_lat': 0.03668090767382201, 
   'advec_lon': -0.16896621297828324, 
   'nugget': 3.0318146866074696e-14}


Final Interpretable Params: {
   'sigma_sq': 12.652739121404919, 
   'range_lon': 0.17664470485305778, 
   'range_lat': 0.14565426376210802, 
   'range_time': 0.8720031476394872, 
   'advec_lat': 0.04466510371597738, 
   'advec_lon': -0.16447983825003892, 'nugget': 0.31139273714919946}



###

Final Interpretable Params: {
   'sigma_sq': 12.600402314947898, 
   'range_lon': 0.15632064225879247, 
   'range_lat': 0.12850412363793687, 
   'range_time': 0.420237386856305, 
   'advec_lat': 3.646629197697307, 
   'advec_lon': -1.82731716498955, 
   'nugget': 3.3125395377840276e-12}


   Final Interpretable Params: {
      'sigma_sq': 12.465255157603531,
       'range_lon': 0.16088131264500707, 
       'range_lat': 0.13795064901185095, 
       'range_time': 0.25642913066711404, 
       'advec_lat': 0.34121830847670426, 
       'advec_lon': -0.9086603984853497, 'nugget': 0.3413704158367853}



###

Final Interpretable Params: {
   'sigma_sq': 12.84409971671357, 
   'range_lon': 0.17337138953730877, 
   'range_lat': 0.13617925813922688, 
   'range_time': 0.08597495864893002,
    'advec_lat': -2.7365555385103155, 
    'advec_lon': -1.3125995387207232, 
    'nugget': 1.4388537434800932e-15}




# TRUE PARAMETERS
init_sigmasq   = 13.059
init_range_lon = 0.195 
init_range_lat = 0.154 
init_advec_lat = 0.0418
init_range_time = 1.0
init_advec_lon = -0.1689
init_nugget    = 0.247


irregular grid

Final Interpretable Params: {
   'sigma_sq': 13.1581147614216, 
   'range_lon': 0.1670857937279293, 
   'range_lat': 0.13532624419142394, 
   'range_time': 0.662083817877393, 
   'advec_lat': 4.485362408040999, 
   'advec_lon': -0.05054465623325784, 
   'nugget': 3.202037263471046e-05}

regular grid

Final Interpretable Params: {
   'sigma_sq': 13.547646279613172, 
   'range_lon': 0.2063608511650159, 
   'range_lat': 0.1698378442441089, 
   'range_time': 1.122662499886297, 
   'advec_lat': 0.05415059834603563, 
   'advec_lon': -0.16640862287821778, 
   'nugget': 0.48795387825462566}




# fit dw

difference data

In [25]:
a = [11.0474, 0.0623, 0.2445, 1.0972, 0.0101, -0.1671, 1.1825]
day = 0 # 0 index
lat_range= [0,5]
lon_range= [123.0, 133.0]
#lat_range= [1,3]
#lon_range= [125, 129.0]

daily_aggregated_tensors_dw = [aggregated_data]
daily_hourly_maps_dw = [input_map]

db = debiased_whittle.debiased_whittle_preprocess(daily_aggregated_tensors_dw, daily_hourly_maps_dw, day_idx=day, params_list=a, lat_range=lat_range, lon_range=lon_range)


subsetted_aggregated_day = db.generate_spatially_filtered_days(0,5,123,133)
print(subsetted_aggregated_day.shape)
N2= subsetted_aggregated_day.shape[0]
print(N2)
subsetted_aggregated_day[:20]

torch.Size([142832, 4])
142832


tensor([[ 2.8000e-02,  1.2305e+02,  0.0000e+00,  2.1000e+01],
        [ 2.8000e-02,  1.2311e+02,  1.9113e+00,  2.1000e+01],
        [ 2.8000e-02,  1.2317e+02,  3.1846e+00,  2.1000e+01],
        [ 2.8000e-02,  1.2323e+02,  9.3044e+00,  2.1000e+01],
        [ 2.8000e-02,  1.2330e+02,  7.5384e+00,  2.1000e+01],
        [ 2.8000e-02,  1.2336e+02, -8.2040e-01,  2.1000e+01],
        [ 2.8000e-02,  1.2342e+02, -1.2852e+00,  2.1000e+01],
        [ 2.8000e-02,  1.2349e+02, -3.2619e+00,  2.1000e+01],
        [ 2.8000e-02,  1.2355e+02, -6.2629e-01,  2.1000e+01],
        [ 2.8000e-02,  1.2361e+02,  1.3124e+00,  2.1000e+01],
        [ 2.8000e-02,  1.2368e+02,  4.2001e-01,  2.1000e+01],
        [ 2.8000e-02,  1.2374e+02, -4.3216e-01,  2.1000e+01],
        [ 2.8000e-02,  1.2380e+02, -5.5636e+00,  2.1000e+01],
        [ 2.8000e-02,  1.2386e+02, -2.0360e+00,  2.1000e+01],
        [ 2.8000e-02,  1.2393e+02,  4.5638e+00,  2.1000e+01],
        [ 2.8000e-02,  1.2399e+02, -5.2148e-02,  2.1000e+01],
        

In [26]:

dwl = debiased_whittle.debiased_whittle_likelihood()
if __name__ == '__main__':
    start_time = time.time()

    # --- Configuration ---
    DAY_TO_RUN = 3 # data is decided above
    TAPERING_FUNC = dwl.cgn_hamming # Use Hamming taper
    NUM_RUNS = 1
    MAX_STEPS = 20 # L-BFGS usually converges in far fewer steps
    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {DEVICE}")

    DELTA_LAT, DELTA_LON = 0.044, 0.063 

    LAT_COL, LON_COL = 0, 1
    VAL_COL = 2 # Spatially differenced value
    TIME_COL = 3


    cur_df =subsetted_aggregated_day
    
    if cur_df.numel() == 0 or cur_df.shape[1] <= max(LAT_COL, LON_COL, VAL_COL, TIME_COL):
        print(f"Error: Data for Day {DAY_TO_RUN} is empty or invalid.")
        exit()

    unique_times = torch.unique(cur_df[:, TIME_COL])
    time_slices_list = [cur_df[cur_df[:, TIME_COL] == t_val] for t_val in unique_times]

    # --- 1. Pre-compute J-vector, Taper Grid, and Taper Autocorrelation ---
    print("Pre-computing J-vector (Hamming taper)...")
    
    # --- 💥 REVISED: Renamed 'p' to 'p_time' 💥 ---
    J_vec, n1, n2, p_time, taper_grid = dwl.generate_Jvector_tapered( 
        time_slices_list,
        tapering_func=TAPERING_FUNC, 
        lat_col=LAT_COL, lon_col=LON_COL, val_col=VAL_COL,
        device=DEVICE
    )

    if J_vec is None or J_vec.numel() == 0 or n1 == 0 or n2 == 0 or p_time == 0:
       print(f"Error: J-vector generation failed for Day {DAY_TO_RUN}.")
       exit()
       
    print("Pre-computing sample periodogram...")
    I_sample = dwl.calculate_sample_periodogram_vectorized(J_vec)

    print("Pre-computing Hamming taper autocorrelation...")
    taper_autocorr_grid = dwl.calculate_taper_autocorrelation_fft(taper_grid, n1, n2, DEVICE)

    if torch.isnan(I_sample).any() or torch.isinf(I_sample).any():
        print("Error: NaN/Inf in sample periodogram.")
        exit()
    if torch.isnan(taper_autocorr_grid).any() or torch.isinf(taper_autocorr_grid).any():
        print("Error: NaN/Inf in taper autocorrelation.")
        exit()

    print(f"Data grid: {n1}x{n2}, {p_time} time points. J-vector, Periodogram, Taper Autocorr on {DEVICE}.")
    # --- END REVISION ---

    # --- 2. Optimization Loop ---
    all_final_results = []
    all_final_losses = []

    for i in range(NUM_RUNS):
        print(f"\n{'='*30} Initialization Run {i+1}/{NUM_RUNS} {'='*30}")

        # --- 7-PARAMETER initialization ---
        ''' 
        init_sigmasq   = 15.0
        init_range_lat = 0.66 
        init_range_lon = 0.7 
        init_nugget    = 1.5
        init_beta      = 0.1  # Temporal range ratio
        init_advec_lat = 0.02
        init_advec_lon = -0.08
        '''
        init_sigmasq   = 13.059
        init_range_lat = 0.154 
        init_range_lon = 0.195
        init_advec_lat = 0.0218
        init_range_time = 0.7
        init_advec_lon = -0.1689
        init_nugget    = 0.247

        init_phi2 = 1.0 / init_range_lon
        init_phi1 = init_sigmasq * init_phi2
        init_phi3 = (init_range_lon / init_range_lat)**2
        # Change needed to match the spatial-temporal distance formula:
        init_phi4 = (init_range_lon / init_range_time)**2      # (range_lon / range_time)^2

        initial_params_values = [
            np.log(init_phi1),    # [0] log_phi1
            np.log(init_phi2),    # [1] log_phi2
            np.log(init_phi3),    # [2] log_phi3
            np.log(init_phi4),    # [3] log_phi4
            init_advec_lat,       # [4] advec_lat (NOT log)
            init_advec_lon,       # [5] advec_lon (NOT log)
            np.log(init_nugget)   # [6] log_nugget
        ]
        
        print(f"Starting with FIXED params (raw log-scale): {[round(p, 4) for p in initial_params_values]}")

        params_list = [
            Parameter(torch.tensor([val], dtype=torch.float64))
            for val in initial_params_values
        ]

        # Helper to define the boundary globally for clarity
        NUGGET_LOWER_BOUND = 0.05
        LOG_NUGGET_LOWER_BOUND = np.log(NUGGET_LOWER_BOUND) # Approx -2.9957

        # --- 💥 REVISED: Use L-BFGS Optimizer 💥 ---
        optimizer = torch.optim.LBFGS(
            params_list,
            lr=1.0,           # Initial step length for line search
            max_iter=20,      # Iterations per step
            history_size=100,
            line_search_fn="strong_wolfe", # Often more robust
            tolerance_grad=1e-5
        )
        # --- END REVISION ---

        print(f"Starting optimization run {i+1} on device {DEVICE} (Hamming, 7-param ST kernel, L-BFGS)...")
        
        # --- 💥 REVISED: Call L-BFGS trainer, pass p_time 💥 ---
        nat_params_str, phi_params_str, raw_params_str, loss, steps_run = dwl.run_lbfgs_tapered(
            params_list=params_list,
            optimizer=optimizer,
            I_sample=I_sample,
            n1=n1, n2=n2, p_time=p_time,
            taper_autocorr_grid=taper_autocorr_grid, 
            max_steps=MAX_STEPS,
            device=DEVICE
        )
        # --- END REVISION ---
        
        if loss is not None:
            all_final_results.append((nat_params_str, phi_params_str, raw_params_str))
            all_final_losses.append(loss)
        else:
            all_final_losses.append(float('inf'))

    print(f"\n\n{'='*25} Overall Result from Run {'='*25} {'='*25}")
    
    valid_losses = [l for l in all_final_losses if l is not None and l != float('inf')]

    if not valid_losses:
        print(f"The run failed or resulted in an invalid loss for Day {DAY_TO_RUN}.")
    else:
        best_loss = min(valid_losses)
        best_run_index = all_final_losses.index(best_loss)
        best_results = all_final_results[best_run_index]
        
        print(f"Best Run Loss: {best_loss} (after {steps_run} steps)")
        print(f"Final Parameters (Natural Scale): {best_results[0]}")
        print(f"Final Parameters (Phi Scale)    : {best_results[1]}")
        print(f"Final Parameters (Raw Log Scale): {best_results[2]}")

    end_time = time.time()
    print(f"\nTotal execution time: {end_time - start_time:.2f} seconds")

Using device: cpu
Pre-computing J-vector (Hamming taper)...
Pre-computing sample periodogram...
Pre-computing Hamming taper autocorrelation...
Data grid: 113x158, 8 time points. J-vector, Periodogram, Taper Autocorr on cpu.

============================== Initialization Run 1/1 ==============================
Starting with FIXED params (raw log-scale): [4.2042, 1.6348, 0.4721, -2.5562, 0.0218, -0.1689, -1.3984]
Starting optimization run 1 on device cpu (Hamming, 7-param ST kernel, L-BFGS)...
--- Step 1/20 ---
 Loss: 1.906929 | Max Grad: 6.274846e-04
  Params (Raw Log): log_phi1: 4.3182, log_phi2: 1.7745, log_phi3: 0.0776, log_phi4: -3.5323, advec_lat: 0.0363, advec_lon: -0.1510, log_nugget: -1.0186
--- Step 2/20 ---
 Loss: 1.817385 | Max Grad: 4.095367e-05
  Params (Raw Log): log_phi1: 4.3182, log_phi2: 1.7744, log_phi3: 0.0775, log_phi4: -3.5320, advec_lat: 0.0363, advec_lon: -0.1510, log_nugget: -1.0177
--- Step 3/20 ---
 Loss: 1.817385 | Max Grad: 4.095367e-05
  Params (Raw Log): log

init_sigmasq   = 13.059
init_range_lon = 0.195 
init_range_lat = 0.154 
init_advec_lat = 0.0418
init_range_time = 1.0
init_advec_lon = -0.1689
init_nugget    = 0.247

1 st simulation (1 vs 3)

Final Interpretable Params: {'sigma_sq': 12.939260896610579, 'range_lon': 0.17242543281507933, 'range_lat': 0.1631231346200311, 'range_time': 1.1358209227858689, 'advec_lat': 0.045119029109988974, 'advec_lon': -0.17809677784942035, 'nugget': 0.3009582276680578}

Final Parameters (Natural Scale): sigmasq: 13.2415, range_lat: 0.1685, range_lon: 0.1739, range_time: 0.9681, advec_lat: 0.0395, advec_lon: -0.1732, nugget: 0.3216

2nd simulation (1 vs 3)

Final Interpretable Params: {'sigma_sq': 12.765293287952144, 'range_lon': 0.17039368470743024, 'range_lat': 0.16152132799710625, 'range_time': 1.081124889959751, 'advec_lat': 0.04635511983762563, 'advec_lon': -0.1775715292039452, 'nugget': 0.31503884896742074}


sigmasq: 13.2415, range_lat: 0.1685, range_lon: 0.1739, range_time: 0.9681, advec_lat: 0.0395, advec_lon: -0.1732, nugget: 0.3216

3nd simulation (mm 15 two times)

Final Interpretable Params: {'sigma_sq': 12.292570063933866, 'range_lon': 0.16170186578939172, 'range_lat': 0.1537053192245325, 'range_time': 0.9671689003322674, 'advec_lat': 0.04155171168950867, 'advec_lon': -0.16218073366456207, 'nugget': 0.29945872751676345}

Final Parameters (Natural Scale): sigmasq: 12.7277, range_lat: 0.1631, range_lon: 0.1696, range_time: 0.9917, advec_lat: 0.0363, advec_lon: -0.1510, nugget: 0.3614
